# Imports

In [ ]:
%pip install tensorflow-data-validation

In [ ]:
%pip install -q tensorflow_data_validation[visualization]

In [15]:
%pip install tfx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.8/436.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 28.4 MB/s eta 0:00:0000:0100:0

UsageError: Line magic function `%tfx` not found.


In [4]:
import sys
import numpy as np
import datetime, calendar
import pandas as pd
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer  
from sklearn import preprocessing
import tensorflow_data_validation as tfdv
np.set_printoptions(threshold=sys.maxsize)
print('TFDV version: {}'.format(tfdv.version.__version__))

# Data loading

In [10]:
TRAIN_DATA = "./data/train.csv"
TEST_DATA = "./data/test.csv"
train_df = pd.read_csv(TRAIN_DATA, sep=";")    
test_df = pd.read_csv(TEST_DATA, sep=";") 
train_df.to_csv(TRAIN_DATA)
test_df.to_csv(TEST_DATA)
display(train_df)
display(train_df.info())

In [11]:
%%capture
import tensorflow_data_validation as tfdv
print('TFDV version: {}'.format(tfdv.version.__version__))
train_stats = tfdv.generate_statistics_from_csv(data_location=TRAIN_DATA)
test_stats = tfdv.generate_statistics_from_csv(data_location=TEST_DATA)

In [12]:
tfdv.visualize_statistics(lhs_statistics=train_stats,
                         rhs_statistics=test_stats)

# Data preprocessing

### Remove ID and TIMESTAMP

In [14]:
train_df.drop(['ID', 'TIMESTAMP'],  axis='columns', inplace=True)
test_df.drop(['ID', 'TIMESTAMP'],  axis='columns', inplace=True)
display(train_df)

,WEBSITE,GDS,DEPARTURE,ARRIVAL,ADULTS,CHILDREN,INFANTS,TRAIN,HAUL_TYPE,DISTANCE,DEVICE,TRIP_TYPE,PRODUCT,SMS,EXTRA_BAGGAGE,NO_GDS
0,EDES,1,22/July,25/July,1,0,0,False,DOMESTIC,"628,844",TABLET,ROUND_TRIP,TRIP,True,False,0
1,EDIT,0,29/July,29/July,1,0,0,False,CONTINENTAL,"1281,43",SMARTPHONE,ONE_WAY,TRIP,False,False,1
2,OPUK,2,29/July,19/August,1,0,0,False,CONTINENTAL,"1730,35",TABLET,ROUND_TRIP,TRIP,True,False,0
3,OPIT,0,24/July,04/August,1,0,0,False,DOMESTIC,"652,702",SMARTPHONE,MULTI_DESTINATION,TRIP,False,False,2
4,EDES,0,11/August,11/August,1,0,0,False,CONTINENTAL,"1717,85",COMPUTER,ONE_WAY,TRIP,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,EDUK,1,02/July,02/July,2,0,0,False,CONTINENTAL,"1035,13",COMPUTER,ONE_WAY,TRIP,True,True,0
49996,EDPT,1,11/August,19/August,2,1,0,False,CONTINENTAL,"1152,2",SMARTPHONE,ROUND_TRIP,TRIP,True,False,0
49997,GOFR,1,09/September,23/September,1,0,0,False,INTERCONTINENTAL,"2236,54",SMARTPHONE,ROUND_TRIP,TRIP,False,False,0
49998,EDPT,2,05/July,21/July,1,0,0,False,CONTINENTAL,"1312,48",COMPUTER,ROUND_TRIP,TRIP,False,True,0


### Remove rows with empty fields

### Transform dates

In [ ]:
month_names = {month.lower(): index for index, month in enumerate(calendar.month_name) if month}
def replaceMonthName(date):
    index = date.find("/")
    month_name = f"{date[index+1:]}".lower()
    return date[:index] + f"{month_names[month_name]}".zfill(2)

def processDates(data):
    data['DEPARTURE'] = data['DEPARTURE'].str.replace("-","/")
    data['ARRIVAL'] = data['ARRIVAL'].str.replace("-","/")
    data['DEPARTURE'] = data['DEPARTURE'].apply(lambda x: f"{replaceMonthName(x)}")
    data['ARRIVAL'] = data['ARRIVAL'].apply(lambda x: f"{replaceMonthName(x)}")
    
processDates(X_train)
processDates(X_test)

### Convert distance to float

In [ ]:
X_train['DISTANCE'] = X_train['DISTANCE'].str.strip()
X_train['DISTANCE'] = X_train['DISTANCE'].apply(lambda x: x.replace(',','.'))
X_test['DISTANCE'] = X_test['DISTANCE'].apply(lambda x: x.replace(',','.'))
X_train['DISTANCE'] = pd.to_numeric(X_train['DISTANCE'], errors='coerce')

### Encode categorical columns

In [ ]:
def hot_encode_categorical_inputs(X_train, X_test, columns):
    oe_style = OneHotEncoder(handle_unknown = 'ignore')   
    
    for col in columns:        
        X_train_enc = oe_style.fit_transform(X_train[[col]])
        X_train = X_train.join(pd.DataFrame(X_train_enc.toarray(), columns=oe_style.categories_))
        X_test_enc = oe_style.transform(X_test[[col]])   
        X_test = X_test.join(pd.DataFrame(X_test_enc.toarray(), columns=oe_style.categories_))
        X_train = X_train.drop([col], axis=1)
        X_test = X_test.drop([col], axis=1)   
    
    return X_train, X_test

def binary_encode_categorical_inputs(X_train, X_test, columns):
    
    for col in columns:   
        X_train[col] = X_train[col].astype(int)
        X_test[col] = X_test[col].astype(int)
        
    return X_train, X_test

hot_encode_cols = ['WEBSITE','DEVICE','HAUL_TYPE','TRIP_TYPE', 'PRODUCT']
label_encode_cols = ['TRAIN', 'SMS']

[X_train, X_test] = hot_encode_categorical_inputs(X_train, X_test, hot_encode_cols)
[X_train, X_test] = binary_encode_categorical_inputs(X_train, X_test, label_encode_cols)

In [ ]:
X_train.dropna(inplace=True)
X_test.dropna(inplace=True)

### Normalize data

In [ ]:
def normalize(X_train, X_test, columns):
    # copy the data
    train = X_train.copy()
    test = X_test.copy()
    normalizer = preprocessing.MinMaxScaler()
    for column in columns:
        train[column] = normalizer.fit_transform(np.array(train[column]).reshape(-1,1))
        test[column] = normalizer.transform(np.array(test[column]).reshape(-1,1))
    return train, test
    
[X_train, X_test] = normalize(X_train, X_test, ['DISTANCE', 'DEPARTURE', 'ARRIVAL'])

### Prepare targets

In [ ]:
def prepare_targets(y_train):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)    
    y_train_enc = pd.DataFrame(y_train_enc)
    return y_train_enc

Y_train = X_train['EXTRA_BAGGAGE']
X_train = X_train.drop('EXTRA_BAGGAGE',  axis='columns')

Y_train = prepare_targets(Y_train)
Y_train.isin([1]).sum()

In [ ]:
saveData("IDs.pickle", X_test["ID"].values)
X_test = X_test.drop('ID',  axis='columns')

# Feature extraction. PCA

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
pca = PCA(n_components=82)
pca.fit(X_train)
fig = plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')
fig.savefig('Cumulative explained variance.png', dpi=fig.dpi)

In [ ]:
pca = PCA(n_components=30)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
pca_std = np.std(X_train)

### Save data

In [ ]:
def saveData(filename, data):
    pickle_out = open(filename, "wb")
    pickle.dump(data, pickle_out)
    pickle_out.close() 


saveData("X_train.pickle", X_train)
saveData("Y_train.pickle", Y_train.values)
saveData("X_test.pickle", X_test)



## Neural network training

In [ ]:
#Dependencies
import time
import tensorflow as tf
import pickle
import numpy as np
import sys
from tf.keras.models import Sequential
from tf.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tf.keras.callbacks import TensorBoard
from tf.keras.callbacks import EarlyStopping


pickle_in = open("X_train.pickle", "rb")
X_train = pickle.load(pickle_in)

pickle_in = open("Y_train.pickle", "rb")
Y_train = pickle.load(pickle_in)
#nonzeroind = np.nonzero(Y_train)[0]
early_stop = EarlyStopping(monitor='val_accuracy', mode='min', patience=50)
NAME = "NN-{}".format(int(time.time()))
tensorBoard = TensorBoard(log_dir='logs/{}'.format(NAME))

model = Sequential()
model.add(Dense(100, input_dim=30, activation='relu')) 
model.add(Dense(50, activation='relu')) 
model.add(Dense(20, activation='relu')) 
model.add(Dense(1, activation='sigmoid'))

model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, Y_train, batch_size=2500, epochs=200,validation_split=0.1, callbacks=[tensorBoard,early_stop], verbose=1,shuffle=True)
model.save('model')


# Model prediction

In [ ]:
from keras.models import load_model
pickle_in = open("X_test.pickle", "rb")
X_test = pickle.load(pickle_in)

pickle_in = open("IDs.pickle", "rb")
IDs = pickle.load(pickle_in)

p = model.predict(X_test)
p = np.array(p).reshape(-1)
IDs = np.array(IDs).reshape(-1)

result = pd.DataFrame({'ID': IDs, 'PREDICTION': p})
result.to_csv('result.csv', index=False)
print(result.head())
print(p[p>0.5].size)